In [ ]:
!echo $http_proxy

In [ ]:
import comet_ml
comet_ml.login()

In [ ]:
from comet_ml import Experiment

In [ ]:
import keras
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.preprocessing import image
from keras.optimizers import RMSprop, Adam,SGD


from keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from keras.callbacks import Callback
import logging
import tensorflow as tf

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255


In [ ]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [ ]:
def build_vanish_grad(input_shape=(28, 28, 1), numnodes=128):
    model = Sequential()

    # Input layer (Flatten the 28x28 image to 784)
    model.add(Flatten(input_shape=input_shape))  # Flatten layer to reshape (28,28,1) to (784,)

    # First hidden layer
    model.add(Dense(numnodes, activation='sigmoid'))

    # Hidden layers: Increasing depth to induce vanishing gradients
    for _ in range(10):  # Add 10 hidden layers
        model.add(Dense(numnodes, activation='sigmoid'))

    # Output layer
    model.add(Dense(10, activation='softmax'))  # Output for 10-class classification

    return model

# Ensure the correct input shape (28, 28, 1) for the image
model = build_vanish_grad(numnodes=128, input_shape=(28, 28, 1))
# Compile with high learning rate and no gradient control
model.compile(optimizer=SGD(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()



In [ ]:
experiment = Experiment(project_name="Vanishing_grad")

In [ ]:
experiment.display()

In [ ]:
class LoggingCallback(Callback):
    def __init__(self, experiment):
        self.experiment = experiment

    def on_epoch_end(self, epoch, logs=None):
        count = 0
        for weight in self.model.get_weights():
            name = "layer-" + str((count // 2) + 1) + "-" + ("bias" if count % 2 == 1 else "weight")
            self.experiment.log_histogram_3d(weight, name=name)
            count += 1
        

In [ ]:
loss_fn = tf.keras.losses.CategoricalCrossentropy()  # Adjust as per your task

# Training loop

for epoch in range(30):
    with tf.GradientTape() as tape:
        predictions = model(x_test)
        loss = loss_fn(y_test, predictions)
    grads = tape.gradient(loss, model.trainable_variables)

            # Log gradients as histograms
    for i, grad in enumerate(grads):
        experiment.log_histogram_3d(
            grad.numpy().flatten(), name=f"Gradient_Layer_{i}", step=epoch
        )

In [ ]:
experiment.log_dataset_hash(x_train)

cb = LoggingCallback(experiment)
with experiment.train():
    model.fit(
        x_train,
        y_train,
        batch_size=32,
        epochs=30,
        callbacks=[cb],
        validation_data=(x_test, y_test)
    )

# callbacks=[cb],
score = model.evaluate(x_test, y_test, verbose=0)
logging.info("Score %s", score)
experiment.end()